### LCZ

In [1]:
# !wget https://zenodo.org/records/8419340/files/lcz_filter_v3.tif?download=1

--2025-01-15 11:41:45--  https://zenodo.org/records/8419340/files/lcz_filter_v3.tif?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1384822872 (1,3G) [image/tiff]
Saving to: ‘lcz_filter_v3.tif?download=1’

lcz_filter_v3.tif?d 100%[===================>]   1,29G  5,49MB/s    in 3m 42s  

2025-01-15 11:45:27 (5,95 MB/s) - ‘lcz_filter_v3.tif?download=1’ saved [1384822872/1384822872]



In [8]:
# !mv lcz_filter_v3.tif /data/uscuni-ulce/validation/

In [66]:
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr
import pandas as pd
import xvec

furl = "/data/uscuni-ulce/validation/lcz_filter_v3.tif"
v = 'v3'

In [2]:
lczs = rioxarray.open_rasterio(furl).squeeze()

In [3]:
lczs

<xarray.DataArray (y: 155995, x: 389620)> Size: 61GB
[60778771900 values with dtype=uint8]
Coordinates:
    band         int64 8B 1
  * x            (x) float64 3MB -170.0 -170.0 -170.0 ... 180.0 180.0 180.0
  * y            (y) float64 1MB 80.04 80.04 80.04 ... -60.09 -60.09 -60.09
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0

In [63]:
regions = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')

In [64]:
%%time

results = []

for region_id in regions.index:

    

    print(region_id)
    buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/clusters_{region_id}_{v}.pq')

    # setup geometry list to burn into the raster
    buildings['num_index'] = np.arange(1, buildings.shape[0] + 1)
    buildings['geometry'] = buildings.representative_point()
    building_utm = buildings.to_crs(lczs.rio.crs)
    minx, miny, maxx, maxy = building_utm.total_bounds


    # read only the region part of the raster data
    region_lczs = rioxarray.open_rasterio(
        filename=furl, 
        chunks='auto', 
        parse_coordinates=True,
        masked=True
    ).rio.clip_box(minx, miny, maxx, maxy)

    # extract building classification from lcz raster
    aggregated_iterative = region_lczs.xvec.extract_points(
        building_utm.geometry,
        x_coords="x",
        y_coords="y",
    )

    # append to the building dataframe and store in the  results array
    res = aggregated_iterative.to_pandas()
    res = gpd.GeoDataFrame(res.T.reset_index(), crs=building_utm.crs)
    res.columns = ['geometry', 'label']
    building_utm['lcz'] = res['label'].values
    results.append(building_utm)

4
10
132
134
286
313
400
523
765
801
832
913
960
1124
1154
1387
1478
1515
1605
1718
1736
1782
1970
1981
2096
2322
2350
2478
2514
2625
2728
2975
3039
3109
3150
3221
3250
3526
3610
3612
3701
3705
3752
3759
3981
4070
4214
4215
4235
4284
4356
4382
4723
4805
5096
5191
5246
5310
5408
5427
5662
5671
5766
5883
6254
6529
6560
6576
6741
6749
6811
6873
6996
7068
7094
7280
7485
7528
7534
7681
7688
7712
7727
7805
7914
7937
7963
8046
8216
8238
8256
8265
8345
8374
8396
8592
8707
8731
8757
8759
8813
9016
9064
9074
9150
9169
9194
9284
9824
9924
9954
9972
10019
10086
10095
10124
10179
10222
10263
10277
10455
10510
10511
10563
10579
10602
10666
10794
10847
10908
10926
10970
11002
11019
11057
11141
11210
11256
11261
11305
11309
11311
11318
11367
11444
11455
11471
11667
11678
11735
11757
11799
11877
11905
12027
12084
12100
12115
12154
12191
12381
12440
12483
12552
12667
12707
12755
12756
12844
12919
12965
13076
13137
13172
13191
13196
13229
13301
13395
13442
13482
13506
13553
13555
13614
13616
13655
13677


In [67]:
all_data = pd.concat(results, ignore_index=True)

In [68]:
cluster_mapping = pd.read_parquet(f'/data/uscuni-ulce/processed_data/clusters/cluster_mapping_{v}.pq')

In [71]:
cluster_mapping

,1,2,3,4,5,6
7,,,,,,
1,1,1,1,1,1,1
2,1,1,1,1,1,1
3,1,1,1,1,1,2
4,1,1,1,1,1,2
5,1,1,1,1,2,3
...,...,...,...,...,...,...
104,2,4,8,16,31,59
105,2,4,8,16,31,59
106,2,4,8,16,32,60


In [73]:
all_data['map_label'] = all_data['final_without_noise'].map(cluster_mapping[4].to_dict())

In [84]:
pd.crosstab(all_data.map_label, all_data.lcz, normalize='index').style.background_gradient(axis=1, cmap="BuGn")

lcz,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000
map_label,,,,,,,,,,,,,,,,,
1,0.000244,0.009800,0.004320,0.003949,0.098718,0.771797,0.000003,0.022409,0.070809,0.000210,0.005038,0.004012,0.000000,0.008615,0.000017,0.000006,0.000054
2,0.007245,0.233480,0.029234,0.029432,0.245145,0.387011,0.000000,0.034139,0.027674,0.002725,0.001234,0.000582,0.000000,0.002001,0.000007,0.000010,0.000080
3,0.148966,0.000000,0.000000,0.008276,0.001379,0.103448,0.000000,0.624828,0.000000,0.110345,0.002759,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.287554,0.014306,0.000000,0.001431,0.000000,0.010014,0.000000,0.682403,0.000000,0.000000,0.000000,0.000000,0.000000,0.001431,0.002861,0.000000,0.000000
5,0.000064,0.000912,0.000364,0.000471,0.049864,0.797926,0.000003,0.009366,0.105799,0.000194,0.012398,0.009342,0.000000,0.013195,0.000000,0.000002,0.000099
6,0.000009,0.000275,0.000384,0.000221,0.021977,0.806686,0.000000,0.005677,0.135083,0.000024,0.006612,0.007749,0.000000,0.015258,0.000003,0.000007,0.000036
7,0.000065,0.000035,0.000049,0.000069,0.007427,0.529825,0.000001,0.004683,0.327147,0.000037,0.037836,0.048059,0.000000,0.044581,0.000026,0.000007,0.000153
8,0.000017,0.000192,0.000162,0.000295,0.011843,0.407233,0.000000,0.010010,0.408081,0.000041,0.029654,0.046727,0.000000,0.085506,0.000084,0.000017,0.000136
9,0.000164,0.000145,0.000043,0.000385,0.000846,0.016929,0.000000,0.008588,0.275312,0.000653,0.067578,0.181480,0.000001,0.446917,0.000502,0.000087,0.000372


In [94]:
# building_utm.iloc[160_000: 190_000].explore(column='lcz', categorical=True, prefer_canvas=True, tiles='cartodb positron')

### Urban atlas

In [1]:
!wget https://copernicus-fme.eea.europa.eu/clmsdatadownload/results/141771.zip

--2025-01-16 10:24:37--  https://copernicus-fme.eea.europa.eu/clmsdatadownload/results/141771.zip
Resolving copernicus-fme.eea.europa.eu (copernicus-fme.eea.europa.eu)... 98.71.17.92
Connecting to copernicus-fme.eea.europa.eu (copernicus-fme.eea.europa.eu)|98.71.17.92|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 340073497 (324M) [application/zip]
Saving to: ‘141771.zip’

141771.zip          100%[===================>] 324,32M  6,93MB/s    in 59s     

2025-01-16 10:25:36 (5,53 MB/s) - ‘141771.zip’ saved [340073497/340073497]



In [1]:
!mv 141771.zip /data/uscuni-ulce/validation/